In [1]:
import tensorflow as tf
from sklearn.datasets import load_digits
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import LabelBinarizer

d:\python\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
d:\python\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# load data
digits = load_digits()
X = digits.data  ##X.shape:(1797, 64)
y = digits.target  ##y.shape:(1797,).. y=array([0, 1, 2, ..., 8, 9, 8])
y = LabelBinarizer().fit_transform(y)   
'''###这个操作是把y改成One——hot类型  y.shape: (1797, 10)   
   y=array([[1, 0, 0, ..., 0, 0, 0],
           [0, 1, 0, ..., 0, 0, 0],
           [0, 0, 1, ..., 0, 0, 0],
                ...,
           [0, 0, 0, ..., 0, 1, 0],
           [0, 0, 0, ..., 0, 0, 1],
           [0, 0, 0, ..., 0, 1, 0]])'''
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3)
'''X_train.shape=(1257, 64)
   y_train.shape=(1257, 10)
   X_test.shape=(540, 64)
   y_test.shape=(540, 10)'''

'X_train.shape=(1257, 64)\n   y_train.shape=(1257, 10)\n   X_test.shape=(540, 64)\n   y_test.shape=(540, 10)'

In [3]:
def add_layer(inputs,in_size,out_size,n_layer,activation_function=None):
    layer_name='layer%s'%n_layer
    with tf.name_scope(layer_name):
        with tf.name_scope('weights'):
            Weights = tf.Variable(tf.random_normal([in_size,out_size],dtype=tf.float32),name='W')##设权重初始值为随机生成的矩阵
            tf.summary.histogram(layer_name+'/weights',Weights)  ##通过histogram显示
        with tf.name_scope('biases'):   
            biases = tf.Variable(tf.zeros([1,out_size])+0.1,name='b')  ##机器学习中推荐偏置初始值不为0,为什么是一行，n_out列，为啥不是反过来？？？
            tf.summary.histogram(layer_name+'/biases',biases)
            ###回答此问题，可以看看下面的公式
        with tf.name_scope('Wx_plus_b'):
            Wx_plus_b = tf.add(tf.matmul(inputs,Weights),biases)### x*W+B
            Wx_plus_b = tf.nn.dropout(Wx_plus_b,keep_prob)
        if activation_function is None:
            outputs = Wx_plus_b
        else:
            outputs = activation_function(Wx_plus_b)
        tf.summary.histogram(layer_name+'/outputs',outputs)###这个貌似是和后面的loss那边的tf.summary.scalar()是配套使用的
        return outputs     

In [4]:
###define placeholder for inputs to network
keep_prob = tf.placeholder(tf.float32)  ###定义丢掉率
xs = tf.placeholder(tf.float32,[None,64])  ###8*8=64,None表示其值大小不定，在这里作为第一个维度值，用以指代batch的大小，意即x的数量不定
ys = tf.placeholder(tf.float32,[None,10])  ##10类输出

In [5]:
##add output layer
l1 = add_layer(xs,64,100,n_layer=1,activation_function=tf.nn.tanh)
prediction = add_layer(l1,100,10,n_layer=2,activation_function = tf.nn.softmax)

In [6]:
##the loss between predition and real data 
cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys*tf.log(prediction),reduction_indices=[1]))##loss
tf.summary.scalar('loss',cross_entropy)
train_step = tf.train.GradientDescentOptimizer(0.6).minimize(cross_entropy)

In [7]:
sess = tf.Session()
merged = tf.summary.merge_all()  ##合并之前的所有变量
train_writer = tf.summary.FileWriter("logs/train",sess.graph)
test_writer = tf.summary.FileWriter("logs/test",sess.graph)
sess.run(tf.global_variables_initializer())

In [8]:
for i in range(1000):
    sess.run(train_step,feed_dict={xs:X_train,ys:y_train,keep_prob:0.5})  ##保持50%的概率不被丢掉。。如果keep_prob=0.6，保持60%的概率不被丢掉，也就是丢掉了40%
    if i % 50 == 0:
        ##record loss
        train_result = sess.run(merged,feed_dict={xs:X_train,ys:y_train,keep_prob:1})##这里我并不想丢掉任何数据
        test_result = sess.run(merged,feed_dict={xs:X_test,ys:y_test,keep_prob:1})
        train_writer.add_summary(train_result,i)
        test_writer.add_summary(test_result,i)           